In [ ]:
#Please run the first 3 cells to have all things to work

!pip install stable-baselines
#!pip install --upgrade pip
!pip install gym[box2d]
!pip install -c conda-forge pyglet
#remove " > /dev/null 2>&1" to see what is going on under the hood
!apt-get install swig cmake libopenmpi-dev zlib1g-dev xvfb x11-utils ffmpeg -qq #remove -qq for full output
!pip install stable-baselines[mpi] box2d box2d-kengz pyvirtualdisplay pyglet==1.3.1 --quiet #remove --quiet for full output 
# Stable Baselines only supports tensorflow 1.x for now
#%tensorflow_version 1.x

import imageio
import base64
import IPython
import PIL.Image
import pyvirtualdisplay
#Video stuff 
from pathlib import Path
from IPython import display as ipythondisplay

In [ ]:
#!sudo apt-get update && sudo apt-get install cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines[mpi]

%tensorflow_version 1.x
#import tensorflow as tf

#!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0

#!pip install gym
#!pip install gym[atari]

#import os
#os.kill(os.getpid(), 9)

import stable_baselines
stable_baselines.__version__

import gym
import numpy as np

In [ ]:
#Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime, os

In [ ]:
#use it when you need to remove old log values and show the new trained one
#!rm -r ppo

# CartPole

In [ ]:
%tensorflow_version 1.x
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


try:
  import optuna
except ImportError:
  !pip install optuna
  import optuna
import gym
import os
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines import PPO2, DQN
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.cmd_util import make_vec_env

# https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb
#from custom_env import GoLeftEnv

def optimize_ppo2(trial):
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': int(trial.suggest_loguniform('n_steps', 16, 2048)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
        'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.4),
        'noptepochs': int(trial.suggest_loguniform('noptepochs', 1, 48)),
        'lam': trial.suggest_uniform('lam', 0.8, 1.),
        #'capacity': int(trial.suggest_loguniform('n_steps', 1, 100))
    }


def optimize_agent(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    # Create log dir
    #log_dir = "tmp/"
    #os.makedirs(log_dir, exist_ok=True)

    env_name='CartPole-v1'
    #env_name='LunarLander-v2'
    #env_name='MountainCar-v0'
    #env_name='Acrobot-v1'
    model_params = optimize_ppo2(trial)
    #env = make_vec_env(lambda: gym.make(env_name), n_envs=16, seed=0)
    env = gym.make(env_name)
    
    model = PPO2('MlpPolicy', env, verbose=0, nminibatches=1, **model_params,
                 tensorboard_log="./cartPole/")
    #model = PPO('MlpPolicy', env, verbose=0, **model_params)
    
    time_steps=20000
    model.learn(total_timesteps=int(time_steps))


    mean_reward, _ = evaluate_policy(model, gym.make(env_name), n_eval_episodes=10)
    #print(model.)
    return mean_reward


if __name__ == '__main__':
    study = optuna.create_study(direction="maximize",pruner=optuna.pruners.SuccessiveHalvingPruner())
    try:
        study.optimize(optimize_agent, n_trials=5, n_jobs=4,)
        #%tensorboard --logdir ./ppo/
    except KeyboardInterrupt:
        print('Interrupted by keyboard.')

[I 2020-12-14 00:44:43,972] A new study created in memory with name: no-name-257673d5-dcec-4651-bed9-6f49b488ba1b
[I 2020-12-14 00:46:31,974] Trial 1 finished with value: 71.4 and parameters: {'n_steps': 606.2417149318093, 'gamma': 0.9838655933179025, 'learning_rate': 0.11311189551565551, 'ent_coef': 2.2419651672376814e-05, 'cliprange': 0.12379496248365413, 'noptepochs': 2.6720941917209076, 'lam': 0.9365721648673035}. Best is trial 1 with value: 71.4.
[I 2020-12-14 00:46:33,417] Trial 2 finished with value: 142.6 and parameters: {'n_steps': 1163.8226312860334, 'gamma': 0.9773512543701407, 'learning_rate': 0.01732035525868406, 'ent_coef': 0.003189465265855713, 'cliprange': 0.3468753061306529, 'noptepochs': 1.3540944866807738, 'lam': 0.9021292541166438}. Best is trial 2 with value: 142.6.
[I 2020-12-14 00:46:41,217] Trial 0 finished with value: 245.7 and parameters: {'n_steps': 1076.3789033095024, 'gamma': 0.9332154594191684, 'learning_rate': 0.003892845648900843, 'ent_coef': 0.028245591

In [ ]:
#Run cartpole
%tensorboard --logdir ./cartPole/

# LunarLander

In [ ]:
def optimize_agent(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    # Create log dir
    #log_dir = "tmp/"
    #os.makedirs(log_dir, exist_ok=True)

    #env_name='CartPole-v1'
    env_name='LunarLander-v2'
    #env_name='MountainCar-v0'
    #env_name='Acrobot-v1'
    model_params = optimize_ppo2(trial)
    #env = make_vec_env(lambda: gym.make(env_name), n_envs=16, seed=0)
    env = gym.make(env_name)
    
    model = PPO2('MlpPolicy', env, verbose=0, nminibatches=1, **model_params,
                 tensorboard_log="./lunarlander/")
    #model = PPO('MlpPolicy', env, verbose=0, **model_params)
    
    time_steps=20000
    model.learn(total_timesteps=int(time_steps))


    mean_reward, _ = evaluate_policy(model, gym.make(env_name), n_eval_episodes=10)
    #print(model.)
    return mean_reward


if __name__ == '__main__':
    study = optuna.create_study(direction="maximize",pruner=optuna.pruners.SuccessiveHalvingPruner())
    try:
        study.optimize(optimize_agent, n_trials=5, n_jobs=4,)
        #%tensorboard --logdir ./lunarlander/
    except KeyboardInterrupt:
        print('Interrupted by keyboard.')

[I 2020-12-14 00:49:36,228] A new study created in memory with name: no-name-6810605e-bdf3-4f04-989f-932b23b7bbe2
[I 2020-12-14 00:49:39,943] Trial 2 finished with value: -105.71759627572624 and parameters: {'n_steps': 845.7774711919889, 'gamma': 0.9384890276983301, 'learning_rate': 0.0005456317266364103, 'ent_coef': 6.446493675195813e-05, 'cliprange': 0.39353368350716467, 'noptepochs': 1.8627910086771269, 'lam': 0.9038914894837258}. Best is trial 2 with value: -105.71759627572624.
[I 2020-12-14 00:49:41,083] Trial 1 finished with value: -841.885245605856 and parameters: {'n_steps': 1036.7215990234245, 'gamma': 0.9937370598806847, 'learning_rate': 0.003544362396638511, 'ent_coef': 1.974202717412899e-06, 'cliprange': 0.21890699218422788, 'noptepochs': 8.899961938379576, 'lam': 0.8711678533635603}. Best is trial 2 with value: -105.71759627572624.
[I 2020-12-14 00:49:54,038] Trial 3 finished with value: -541.6870796397176 and parameters: {'n_steps': 17.91650006366898, 'gamma': 0.915844160

In [ ]:
#Run lunarLander
%tensorboard --logdir ./lunarlander/

# MountainCar

In [ ]:
def optimize_agent(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    # Create log dir
    #log_dir = "tmp/"
    #os.makedirs(log_dir, exist_ok=True)

    #env_name='CartPole-v1'
    #env_name='LunarLander-v2'
    env_name='MountainCar-v0'
    #env_name='Acrobot-v1'
    model_params = optimize_ppo2(trial)
    #env = make_vec_env(lambda: gym.make(env_name), n_envs=16, seed=0)
    env = gym.make(env_name)
    
    model = PPO2('MlpPolicy', env, verbose=0, nminibatches=1, **model_params,
                 tensorboard_log="./MountainCar/")
    #model = PPO('MlpPolicy', env, verbose=0, **model_params)
    
    time_steps=20000
    model.learn(total_timesteps=int(time_steps))


    mean_reward, _ = evaluate_policy(model, gym.make(env_name), n_eval_episodes=10)
    #print(model.)
    return mean_reward


if __name__ == '__main__':
    study = optuna.create_study(direction="maximize",pruner=optuna.pruners.SuccessiveHalvingPruner())
    try:
        study.optimize(optimize_agent, n_trials=5, n_jobs=4,)
        #%tensorboard --logdir ./MountainCar/
    except KeyboardInterrupt:
        print('Interrupted by keyboard.')

[I 2020-12-14 00:59:11,363] A new study created in memory with name: no-name-d2f02182-d25d-4a6a-8d96-600d4dfa6652
[I 2020-12-14 01:02:16,079] Trial 0 finished with value: -200.0 and parameters: {'n_steps': 438.1320239158379, 'gamma': 0.976888084210343, 'learning_rate': 0.008569495909565107, 'ent_coef': 1.6154532007233593e-07, 'cliprange': 0.1553591451650541, 'noptepochs': 1.5760136431055483, 'lam': 0.8285594283487052}. Best is trial 0 with value: -200.0.
[I 2020-12-14 01:03:29,875] Trial 1 finished with value: -200.0 and parameters: {'n_steps': 20.050674451352126, 'gamma': 0.9311514871252871, 'learning_rate': 1.8929888474605957e-05, 'ent_coef': 0.004526360224804509, 'cliprange': 0.2257727903029321, 'noptepochs': 3.1186497526629986, 'lam': 0.8980575036063343}. Best is trial 0 with value: -200.0.
[I 2020-12-14 01:06:36,978] Trial 3 finished with value: -200.0 and parameters: {'n_steps': 22.35401699014769, 'gamma': 0.9872203601975728, 'learning_rate': 0.00028540999663384504, 'ent_coef': 0

In [ ]:
#Run lunarLander
%tensorboard --logdir ./MountainCar/

# Acrobot

In [ ]:
def optimize_agent(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    # Create log dir
    #log_dir = "tmp/"
    #os.makedirs(log_dir, exist_ok=True)

    #env_name='CartPole-v1'
    #env_name='LunarLander-v2'
    #env_name='MountainCar-v0'
    env_name='Acrobot-v1'
    model_params = optimize_ppo2(trial)
    #env = make_vec_env(lambda: gym.make(env_name), n_envs=16, seed=0)
    env = gym.make(env_name)
    
    model = PPO2('MlpPolicy', env, verbose=0, nminibatches=1, **model_params,
                 tensorboard_log="./Acrobot/")
    #model = PPO('MlpPolicy', env, verbose=0, **model_params)
    
    time_steps=20000
    model.learn(total_timesteps=int(time_steps))


    mean_reward, _ = evaluate_policy(model, gym.make(env_name), n_eval_episodes=10)
    #print(model.)
    return mean_reward


if __name__ == '__main__':
    study = optuna.create_study(direction="maximize",pruner=optuna.pruners.SuccessiveHalvingPruner())
    try:
        study.optimize(optimize_agent, n_trials=5, n_jobs=4,)
        #%tensorboard --logdir ./Acrobot/
    except KeyboardInterrupt:
        print('Interrupted by keyboard.')

[I 2020-12-14 01:09:34,879] A new study created in memory with name: no-name-38d90f56-b893-4300-8046-114eb1a7a3c4
[I 2020-12-14 01:12:27,343] Trial 1 finished with value: -85.5 and parameters: {'n_steps': 143.11399864802289, 'gamma': 0.9041878292971955, 'learning_rate': 0.0004399699396134245, 'ent_coef': 0.002270250650321917, 'cliprange': 0.16669138402086894, 'noptepochs': 8.177554024377828, 'lam': 0.8592372786661162}. Best is trial 1 with value: -85.5.
[I 2020-12-14 01:12:31,620] Trial 2 finished with value: -85.0 and parameters: {'n_steps': 80.2107402012829, 'gamma': 0.9576391291672102, 'learning_rate': 0.0010047547841189627, 'ent_coef': 7.897429165979982e-05, 'cliprange': 0.1960061897306572, 'noptepochs': 5.472860484783422, 'lam': 0.993380985270454}. Best is trial 2 with value: -85.0.
[I 2020-12-14 01:12:31,885] Trial 0 finished with value: -500.0 and parameters: {'n_steps': 1696.2915208698632, 'gamma': 0.9993149312049932, 'learning_rate': 2.0492263494566494e-05, 'ent_coef': 8.52867

In [ ]:
#Run lunarLander
%tensorboard --logdir ./Acrobot/